In [1]:
import pandas as pd
import os
import bisect
import re

os.getcwd()

'C:\\Users\\ygema\\venv\\notebooks'

In [2]:
f = open("/Users/ygema/Downloads/WhatsApp Chat with אופקים חוגגים שנה חדשה.txt", "r", encoding='utf-8')

conversation_string = f.readlines()

In [3]:
conversation_string[:5]

['2/12/20, 2:37\u202fPM - +972 54-527-7779: <Media omitted>\n',
 '2/12/20, 2:37\u202fPM - +972 54-527-7779: בבקשה\n',
 '2/12/20, 2:37\u202fPM - +972 54-527-7779: איפה אחי.. קניתי סים מקומי\n',
 '2/12/20, 2:37\u202fPM - +972 54-803-0546: חסכת לי ילד עכשיו\n',
 '2/12/20, 2:38\u202fPM - +972 54-527-7779: חחחחחחחח\n']

In [4]:
import re

def find_end_of_questions(x, start_row):
    for idx,item in enumerate(x[start_row:]):
        if re.search(r'\d{1,2}/\d{1,2}/\d{2}', item):
            return start_row + idx

In [5]:
question_start_row = [(idx+1) for idx,val in enumerate(conversation_string) if "שאלות לפרשת" in val]
answer_start_row = [idx+1 for idx,val in enumerate(conversation_string) if "תשובות לפרשת" in val or "תשובות  לפרשת" in val]

In [6]:
question_end_row = [find_end_of_questions(conversation_string,idx) for idx in question_start_row]
answer_end_row = [find_end_of_questions(conversation_string,idx) for idx in answer_start_row]

In [7]:
question_bank = []
for x in zip(question_start_row,question_end_row):
    question_bank.append(conversation_string[(x[0]-1):x[1]])

    
answer_bank = []
for x in zip(answer_start_row,answer_end_row):
    answer_bank.append(conversation_string[(x[0]-1):x[1]])

In [8]:
def clean_up(x):
    cleaned_up_list = []
    for row in x:
        cleaned_up_row = (row.replace("\t",""))
        if cleaned_up_row != '':
            cleaned_up_list.append(cleaned_up_row)
    return cleaned_up_list

In [9]:
def extract_questions(x):
    questions_list = [(row.replace("\n","").replace("\'","")) for idx, row in enumerate(x)]
    first_row = [re.findall(r'(?<=:)(.*?)(?=:)', questions_list[0])[-1].strip()]
    first_row.extend(" ")
    first_row.extend([f'\u202E{row}' for idx, row in enumerate(questions_list[1:]) if re.match(r'[0-9]',row)])
    return first_row

def extract_answers(x):
    answers_list = [(row.replace("\n","").replace("\'","")) for idx, row in enumerate(x) if row.startswith("תש:") or "תשובות לפרשת" in row]
    first_row = [re.findall(r'(?<=:)(.*?)(?=:)', answers_list[0])[-1].strip()]
    first_row.extend(" ")
    first_row.extend([f'\u202E{idx+1}. {row}'.replace("\u202E0.","").replace("תש: ","") for idx, row in enumerate(answers_list[1:])])
    answers_list_enumerated = [x.encode('utf-8').decode('utf-8') for x in first_row]
    return answers_list_enumerated

In [10]:
cleaned_up_question_bank = [clean_up(question) for question in question_bank]
cleaned_up_answer_bank = [clean_up(answer) for answer in answer_bank]

In [11]:
len(cleaned_up_answer_bank)

75

In [12]:
cleaned_up_question_bank_extracted = ["\n".join(extract_questions(question)) for question in cleaned_up_question_bank]
cleaned_up_answer_bank_extracted = ["\n".join(extract_answers(answer)) for answer in cleaned_up_answer_bank]

# to capture all of 5783
cleaned_up_question_bank_extracted = cleaned_up_question_bank_extracted[37:]
cleaned_up_answer_bank_extracted = cleaned_up_answer_bank_extracted[30:]

In [13]:
###################

In [17]:
cleaned_up_question_bank_extracted[-1]

'שאלות לפרשת האזינו\n \n\u202e1. משה וישעיה, שני הפכים. \n\u202e2. מהם שלושת הסיבובים שסיבב הקב"ה? \n\u202e3. קללה שהיא סוג של ברכה \n\u202e4. קללה שמתפרשת ע"י מצוה \n\u202e5. ביטוי למשפט, המופיע בפיוט יום ליבשה. \n\u202e6. ביטוי לרפיון, חולשה \n\u202e7. דמות בספר בראשית המלמדת , שאין דבר שאין בו מה ללמוד'

In [54]:
# numbers are coming out backwards.
# This searches for them to change manually 
# run for answers and for questions

for q in cleaned_up_answer_bank_extracted:
    for line in ((q.split("\n"))):
        if re.search("\d{2,}",line):
            print(line)

‮2. רש"י בשם מדרש רבה אומר שנח נשא את נעמה אחות תובל קין. אם הפסוקים הם כפשוטם- היא נולדה לפני ששת נולד. כלומר היתה גדולה מבעלה בלפחות 900 שנה (אלא אם כן נאמר שהפסוקים לא כפשוטם והיא נולדה ללמך אחרי מאות שנים) 
‮6. למך- שנפטר כשהיה בן 777.
‮3. הנושא אשה ולא העמיד ילדים 10 שנים חייב לשאת אשה אחרת (יבמות סד.)
‮1. 11. יצחק, מואב, עמון, 8 בני מלכה.
‮6. אסתר שמלכה על 127 מדינות.
‮1. שם בן נח, שנפטר כאשר יעקב היה בן 50, בתחילת הפרשה כאשר רבקה הלכה לדרוש את ה נאמר ברש"י שהלכה לבית מדרשו של שם. אך בסוף הפרשה כאשר רש"י מסביר שיעקב למד תורה י"ד שנים הוא אומר שיעקב נטמן בבית עבר. 
‮8. כתוב במדרש שהיתה בתו של שם. שם נפטר 36 שנים בערך לפני שנולד יהודה. אלא אם כן נאמר שלא היתה בתו ממש אלא נכדתו וכד.
‮3. רדו, כמנין 210.
‮4.  יעקב אבינו נענש על כך שאמר לפרעה "מעט ורעים וכו ועל כך ירדו לו 33 שנים כמנין המילים בפסוקים (מדרש רבה) שהיה אמור לחיות 180 שנה כאביו וחי 147. ישנה עוד תשובה  על פי בעל הטורים שאומר שיעקב נענש על כך שבגלל קללתו מתה רחל , ואמר שמי שגנב את התרפים לא יחיה , יחיה בגימטריה 33.
‮1. יעקב

In [14]:
####################

In [21]:
from docx import Document
from docx.shared import Pt
# Sample list of lists


# Create a new Word document
doc = Document()

# Iterate through the list of lists and add each list to a separate page
for i in range(len(cleaned_up_question_bank_extracted)):
    # Create a new page (section break)
    paragraph = doc.add_paragraph()
    
    bold_run = paragraph.add_run((cleaned_up_question_bank_extracted[i]).split("\n")[0])
    bold_run.bold = True
    bold_run.underline = True
    bold_run.font.size = Pt(20)
    
    rest_paragraph_run = paragraph.add_run('\n'.join((cleaned_up_question_bank_extracted[i]).split("\n")[1:]))
    font = rest_paragraph_run.font
    font.size = Pt(20)
    
    doc.add_paragraph().add_run('\n\n\n')
    
    answers_ref_run = doc.add_paragraph().add_run(f'(התשובות בעמוד {i+49})')
    font = answers_ref_run.font
    font.size = Pt(20)
    
    
    doc.add_section()

for i in range(len(cleaned_up_answer_bank_extracted)):
    # Create a new page (section break)
    paragraph = doc.add_paragraph()
    
    bold_run = paragraph.add_run((cleaned_up_answer_bank_extracted[i]).split("\n")[0])
    bold_run.bold = True
    bold_run.underline = True
    bold_run.font.size = Pt(20)
    
    ans_paragraph_run = paragraph.add_run('\n'.join((cleaned_up_answer_bank_extracted[i]).split("\n")[1:]))
    ans_paragraph_run.font.size = Pt(20)
    doc.add_section()

# Save the Word document
doc.save("output.docx")

AttributeError: 'Paragraph' object has no attribute 'font'

In [31]:
from docx import Document
from docx.shared import Pt
# Sample list of lists


# Create a new Word document
doc = Document()

# Iterate through the list of lists and add each list to a separate page
for i in range(len(cleaned_up_question_bank_extracted)):
    # Create a new page (section break)
    paragraph = doc.add_paragraph()
    
    bold_run = paragraph.add_run((cleaned_up_question_bank_extracted[i]).split("\n")[0])
    bold_run.bold = True
    bold_run.underline = True
    bold_run.font.size = Pt(20)
    
    rest_paragraph_run = paragraph.add_run('\n'.join((cleaned_up_question_bank_extracted[i]).split("\n")[1:]))
    font = rest_paragraph_run.font
    font.size = Pt(20)
    
    doc.add_paragraph().add_run('\n\n\n')
    
    answers_ref_run = doc.add_paragraph().add_run(f'(התשובות בעמוד {i+49})')
    font = answers_ref_run.font
    font.size = Pt(20)
    
    
    doc.add_section()

for i in range(len(cleaned_up_answer_bank_extracted)):
    # Create a new page (section break)
    paragraph = doc.add_paragraph()
    
    bold_run = paragraph.add_run((cleaned_up_answer_bank_extracted[i]).split("\n")[0])
    bold_run.bold = True
    bold_run.underline = True
    bold_run.font.size = Pt(20)
    
    ans_paragraph_run = paragraph.add_run('\n'.join((cleaned_up_answer_bank_extracted[i]).split("\n")[1:]))
    ans_paragraph_run.font.size = Pt(20)
    doc.add_section()

# Save the Word document
doc.save("output.docx")

In [29]:
cleaned_up_question_bank_extracted[:5]

['שאלות לפרשת בראשית\n \n\u202e1.  (כיצד קשור המן לפרשה? (שני אופנים\n\u202e2. מי נשא (על פי המדרש) אשה זקנה ממנו בהרבה? \n\u202e3. מי עשוי כאוצר של חיטים? \n\u202e4. מי ענתה "מה מועיל נר בצהריים" ? \n\u202e5. איזו אות נאמרה אחרונה מכל האותיות ? מדוע? \n\u202e6. מה משותף לאדם, ירד ומתושלח? \n\u202e7. למי יש רק ספרה אחת בשנות חייו?\n\u202e8. היכן ספרד מוזכרת בפרשה? \n\u202e8. את שניהם אמר אדם הראשון. במהלך החיים נאמרים על פי הסדר, אך לא כתובים כסדר.',
 'שאלות לפרשת נח\n \n\u202e1. פיוט מזמירות שבת\n\u202e2. אלו שמונה אותיות מיותרות לכאורה יש בפרשה, ומדוע? \n\u202e3. עיר ומלכות, שכינויה בקינות רמוז בפרשה.\n\u202e4. שבעה הם, והשביעי מתוכם התחדש בפרשתנו. \n\u202e5. מי בפרשה עשה קל וחומר? \n\u202e6. איזה כלל בהלכות נימוסים ודרך ארץ נלמד מהפרשה ? \n\u202e7. שנים בעלי שם אחד נשלחו, האחד בפרשתנו, השני, נשלח לעיר מפרשתנו.\n\u202e8. מתי רבים נכתבו בלשון יחיד, ומדוע? ',
 'שאלות לפרשת לך לך\n \n\u202e1. מאורע מהפרשה, מוזכר בפיוט הנאמר פעם בשנה\n\u202e2. מי פחד מהפליט? \n\u202e3. איזו הלכה למדים מנ

In [698]:
from docx import Document
from docx.shared import Inches, Pt

# Create a new Word document
doc = Document()

# Content for the top part of the page
top_content = "This is content near the top of the page."
top_paragraph = doc.add_paragraph(top_content)

# Content for the bottom part of the page
bottom_content = "This is content near the bottom of the page."
bottom_paragraph = doc.add_paragraph(bottom_content)

# Calculate the length of the top and bottom content
top_content_length = len(top_content)
bottom_content_length = len(bottom_content)

# Get the font size of the paragraph (assuming uniform font size)
font_size_pts = top_paragraph.style.font.size.pt

# Calculate the spacing required to align the bottom content with the bottom of the page
spacing_for_bottom = (top_content_length - bottom_content_length) * (font_size_pts / 2)

# Set spacing before the bottom paragraph to move it up the page
bottom_paragraph_format = bottom_paragraph.paragraph_format
bottom_paragraph_format.space_before = Pt(spacing_for_bottom)

# Save the Word document
doc.save("output.docx")

AttributeError: 'NoneType' object has no attribute 'pt'

In [699]:
from docx import Document

# Create a new Word document
doc = Document()

# Create a paragraph
paragraph = doc.add_paragraph()

# Add a run for the regular text
paragraph.add_run("This is regular text. ")

# Add a run for the bold text
bold_run = paragraph.add_run("This is bold text.")
bold_run.bold = True

# Save the Word document
doc.save("output.docx")

In [701]:
cleaned_up_question_bank_extracted[0]

'שאלות לפרשת בראשית\n \n\u202e1. כיצד קשור המן\n\u202e2. מי נשא (על פי המדרש) אשה זקנה ממנו בהרבה? \n\u202e3. מי עשוי כאוצר של חיטים? \n\u202e4. מי ענתה "מה מועיל נר בצהריים" ? \n\u202e5. איזו אות נאמרה אחרונה מכל האותיות ? מדוע? \n\u202e6. מה משותף לאדם, ירד ומתושלח? \n\u202e7. למי יש רק ספרה אחת בשנות חייו?\n\u202e8. היכן ספרד מוזכרת בפרשה? \n\u202e8. את שניהם אמר אדם הראשון. במהלך החיים נאמרים על פי הסדר, אך לא כתובים כסדר.'

In [704]:
(cleaned_up_question_bank_extracted[0]).split("\n")[0]

'שאלות לפרשת בראשית'

In [715]:
''.join((cleaned_up_question_bank_extracted[0]).split("\n")[1:])

' \u202e1. כיצד קשור המן\u202e2. מי נשא (על פי המדרש) אשה זקנה ממנו בהרבה? \u202e3. מי עשוי כאוצר של חיטים? \u202e4. מי ענתה "מה מועיל נר בצהריים" ? \u202e5. איזו אות נאמרה אחרונה מכל האותיות ? מדוע? \u202e6. מה משותף לאדם, ירד ומתושלח? \u202e7. למי יש רק ספרה אחת בשנות חייו?\u202e8. היכן ספרד מוזכרת בפרשה? \u202e8. את שניהם אמר אדם הראשון. במהלך החיים נאמרים על פי הסדר, אך לא כתובים כסדר.'

In [648]:
qs = [i.split("\n")[0] for i in cleaned_up_question_bank_extracted]
ans = [i.split("\n")[0] for i in cleaned_up_answer_bank_extracted]

In [670]:
qs[37:]

['שאלות לפרשת בראשית',
 'שאלות לפרשת נח',
 'שאלות לפרשת לך לך',
 'שאלות לפרשת וירא',
 'שאלות לפרשת חיי שרה',
 'שאלות לפרשת תולדות',
 'שאלות לפרשת ויצא',
 'שאלות לפרשת וישלח',
 'שאלות לפרשת וישב',
 'שאלות לפרשת מקץ',
 'שאלות לפרשת ויגש',
 'שאלות לפרשת ויחי',
 'שאלות לפרשת שמות',
 'שאלות לפרשת וארא',
 'שאלות לפרשת בא',
 'שאלות לפרשת בשלח',
 'שאלות לפרשת יתרו',
 'שאלות לפרשת משפטים',
 'שאלות לפרשת תרומה',
 'שאלות לפרשת תצוה-זכור-פורים',
 'שאלות לפרשת כי תשא-פרה',
 'שאלות לפרשת ויקהל-פקודי-החודש',
 'שאלות לפרשת ויקרא',
 'שאלות לפרשת צו-הגדול',
 'שאלות לפרשת שמיני',
 'שאלות לפרשת תזריע מצורע',
 'שאלות לפרשת אחרי מות קדושים',
 'שאלות לפרשת אמור',
 'שאלות לפרשת בהר-בחוקותי',
 'שאלות לפרשת במדבר',
 'שאלות לפרשת בהעלותך',
 'שאלות לפרשת שלח',
 'שאלות לפרשת קרח',
 'שאלות לפרשת חוקת',
 'שאלות לפרשת בלק',
 'שאלות לפרשת פנחס',
 'שאלות לפרשת מטות מסעי',
 'שאלות לפרשת דברים',
 'שאלות לפרשת ואתחנן - נחמו',
 'שאלות לפרשת עקב',
 'שאלות לפרשת ראה',
 'שאלות לפרשת שופטים',
 'שאלות לפרשת כי תצא',
 'שאלות לפר

In [664]:
ans[30:]

['תשובות לפרשת בראשית',
 'תשובות לפרשת נח',
 'תשובות לפרשת לך לך',
 'תשובות לפרשת וירא',
 'תשובות לפרשת חיי שרה',
 'תשובות לפרשת תולדות',
 'תשובות לפרשת ויצא',
 'תשובות לפרשת וישלח',
 'תשובות לפרשת וישב',
 'תשובות לפרשת מקץ',
 'תשובות לפרשת ויגש',
 'תשובות לפרשת ויחי',
 'תשובות לפרשת שמות',
 'תשובות לפרשת וארא',
 'תשובות לפרשת בא',
 'תשובות לפרשת בשלח',
 'תשובות לפרשת יתרו',
 'תשובות לפרשת משפטים',
 'תשובות לפרשת תרומה',
 'תשובות לפרשת תצוה-זכור-פורים',
 'תשובות לפרשת כי תשא-פרה',
 'תשובות לפרשת ויקהל-פקודי-החודש',
 'תשובות לפרשת ויקרא',
 'תשובות לפרשת צו-הגדול',
 'תשובות לפרשת שמיני',
 'תשובות לפרשת תזריע מצורע',
 'תשובות לפרשת אחרי מות קדושים',
 'תשובות לפרשת אמור',
 'תשובות לפרשת בהר-בחוקותי',
 'תשובות לפרשת במדבר',
 'תשובות לפרשת בהעלותך',
 'תשובות לפרשת שלח',
 'תשובות לפרשת קרח',
 'תשובות לפרשת חוקת',
 'תשובות לפרשת בלק',
 'תשובות לפרשת פנחס',
 'תשובות לפרשת מטות מסעי',
 'תשובות לפרשת דברים',
 'תשובות לפרשת ואתחנן - נחמו',
 'תשובות לפרשת עקב',
 'תשובות לפרשת ראה',
 'תשובות לפרשת ש

In [659]:
len(qs)

83

In [660]:
len(ans)

75

In [656]:
import re

def find_end_of_just_questions(x, start_row):
    for idx,item in enumerate(x[start_row:]):
        if re.search(r'\d{1,2}/\d{1,2}/\d{2}', item):
            return start_row + idx

In [651]:
find_end_of_just_questions(conversation_string,question_start_row[0])

29023

In [654]:
question_start_row[]

SyntaxError: invalid syntax (198091019.py, line 1)

In [ ]:
answer_start_row[0]

In [ ]:
qs[2]

In [ ]:
ans[0]

In [ ]:
qs[5]

In [ ]:
dict(enumerate(ans))

In [ ]:
dict(enumerate(qs))

In [ ]:
len(cleaned_up_question_bank_extracted[15].split("\n"))

In [47]:
re.search(r'\d{2,}','123')

<re.Match object; span=(0, 3), match='123'>

In [ ]:
#########3 notes:

# had some cases where תשובות  לפרשות with 2 spaces.
# other cases had שאלות instead of תשובות

In [ ]:
from docx import Document

# Sample list of lists


# Create a new Word document
doc = Document()

# Iterate through the list of lists and add each list to a separate page
for i in range(len(cleaned_up_question_bank_extracted[:-1])):
    # Create a new page (section break)
    
    doc.add_paragraph(cleaned_up_question_bank_extracted[i])
    doc.add_paragraph(cleaned_up_answer_bank_extracted[i])
    
    doc.add_section()

# Save the Word document
doc.save("output.docx")